In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
import pyttsx3
import speech_recognition as sr
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load your JSON dataset
with open('intents.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data['intents'])

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=2)

# Extract patterns and corresponding tags from the training data
train_patterns = train_data['patterns'].apply(lambda x: ' '.join(x))
train_tags = train_data['tag']

# Initialize CountVectorizer to convert patterns into a matrix of token counts
#feature extraction
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_patterns)

# Initialize a Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB(alpha=0.9)

# Train the model
naive_bayes_classifier.fit(X_train, train_tags)

# Extract patterns and corresponding tags from the testing data
test_patterns = test_data['patterns'].apply(lambda x: ' '.join(x))
test_tags = test_data['tag']

# Transform test patterns using the same vectorizer
X_test = vectorizer.transform(test_patterns)

# Predict tags for test patterns
predicted_tags = naive_bayes_classifier.predict(X_test)
# Initialize the recognizer 
r = sr.Recognizer() 



    
while True:
    with sr.Microphone() as source2:
			
        # wait for a second to let the recognizer
        # adjust the energy threshold based on
        # the surrounding noise level
        print("Speak now")
        r.adjust_for_ambient_noise(source2, duration=0.2)

        #listens for the user's input 
        audio2 = r.listen(source2)

        # Using google to recognize audio
        MyText = r.recognize_google(audio2)
        
        MyText = MyText.lower()

        user_input = MyText
        user_input_vec = vectorizer.transform([user_input])
        predicted_intent = naive_bayes_classifier.predict(user_input_vec)[0]

        # Get the corresponding response based on the predicted intent
        responses = df[df['tag'] == predicted_intent]['responses'].values[0]

        # Select a single response randomly
        response = random.choice(responses)
        print(f"Chatbot: {response}")
        engine = pyttsx3.init()
        
        engine.setProperty('rate', 150)
        engine.say(response)
        engine.runAndWait()

        
